In [2]:
# считывает MovieLens
from os import path
import pandas as pd

data_dir = "/home/ubuntu/data/movielens/ml-latest-small"

def read_csv(filename: str):
    data = pd.read_csv(path.join(data_dir, filename + ".csv"))
    return data

ratings = read_csv("ratings")
movies = read_csv("movies")

In [3]:
# перекодируем id с пробелами в плотные
ratings["movie_id"] = ratings["movieId"].astype("category").cat.codes.copy()
ratings["user_id"] = ratings["userId"].astype("category").cat.codes.copy()

In [4]:
# переводим матрицу взаимодействий пользователей и объектов к разреженному виду
from scipy.sparse import csr_matrix
import numpy as np

last_movie_id = ratings["movie_id"].max()
last_user_id = ratings["user_id"].max()
def sparse_info(sparse_matrix: csr_matrix):
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )

user_x_item = ratings[["user_id", "movie_id"]].as_matrix()
user_x_item
user_item_matrix = csr_matrix(
    (
        ratings["rating"].tolist(),
        (
            [pair[0] for pair in user_x_item],
            [pair[1] for pair in user_x_item],
        )
    ),
    shape=(last_user_id + 1, last_movie_id + 1),
    dtype=np.float32
)

sparse_info(user_item_matrix)

Размерности матрицы: (671, 9066)
Ненулевых элементов в матрице: 100004
Доля ненулевых элементов: 0.016439141608663475


In [5]:
# разобьём наблюдения на тестовую и обучающую выборки
np.random.seed(0)
test_indices = np.random.choice(
    range(user_item_matrix.nnz),
    replace=False,
    size=int(user_item_matrix.nnz * 0.2)
).tolist()
train_data = user_item_matrix.copy()
train_data.data[test_indices] = 0
train_data.eliminate_zeros()
print("размер обучающей выборки: {}".format(train_data.nnz))
test_data = user_item_matrix.copy()
test_data.data[:] = 0
test_data.data[test_indices] = user_item_matrix.data[test_indices]
test_data.eliminate_zeros()
print("размер тестовой выборки: {}".format(test_data.nnz))

размер обучающей выборки: 80004
размер тестовой выборки: 20000


In [6]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, reciprocal_rank

# обучаем абы какую модель
model = LightFM(loss="bpr")
model.fit(
    train_data,
    num_threads=4
)

train_mrr = reciprocal_rank(model, train_data).mean()
test_mrr = reciprocal_rank(model, test_data).mean()
print('MRR: train %.2f, test %.2f.' % (train_mrr, test_mrr))
for k in [5, 10, 15, 20]:
    train_precision = precision_at_k(model, train_data, k=k).mean()
    test_precision = precision_at_k(model, test_data, k=k).mean()
    print('Precision@%d: train %.2f, test %.2f.' % (k, train_precision, test_precision))

ImportError: No module named 'lightfm'

In [ ]:
# выберем информацию по жанрам фильмов
movies_genres = ratings[["movieId", "movie_id"]].drop_duplicates().join(
    movies,
    on="movieId",
    rsuffix="codes",
    lsuffix="movies",
    sort=True
).fillna("None")[["movie_id", "genres"]]
movies_genres["genres_set"] = movies_genres["genres"].apply(lambda x: set(x.split("|")))
movies_genres.head()

In [ ]:
# все возможные жанры
from functools import reduce

reduce(lambda acc, ele: acc.union(ele), movies_genres["genres_set"].tolist(), set())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# приравняем None и (no genres listed)
movies_genres.loc[movies_genres["genres"] == "(no genres listed)", "genres"] = "None"
# уберём все спецсимволы, кроме |
movies_genres["genres"] = movies_genres["genres"].apply(
    lambda x: x.replace("-", "")
)

movies_features = CountVectorizer().fit_transform(movies_genres["genres"])
movies_features

In [ ]:
print(movies_genres["genres"][:1])
print(movies_features[0].todense())

In [ ]:
# добавляем к movie_id ещё и информацию о жанрах
from scipy.sparse import hstack, identity

features = hstack([
    identity(movies_genres.shape[0]),
    movies_features
])

In [7]:
# обучаем модель с жанрами
hybrid = LightFM(loss="bpr")
hybrid.fit(
    train_data,
    num_threads=4,
    item_features=features
)

train_mrr = reciprocal_rank(hybrid, train_data, item_features=features).mean()
test_mrr = reciprocal_rank(hybrid, test_data, item_features=features).mean()
print('MRR: train %.2f, test %.2f.' % (train_mrr, test_mrr))
for k in [5, 10, 15, 20]:
    train_precision = precision_at_k(hybrid, train_data, item_features=features, k=k).mean()
    test_precision = precision_at_k(hybrid, test_data, item_features=features, k=k).mean()
    print('Precision@%d: train %.2f, test %.2f.' % (k, train_precision, test_precision))

NameError: name 'LightFM' is not defined